# Business Optimization: Production Planning with Linear Programming

## Problem Overview

**Business Context:**  
A furniture manufacturing company produces three types of products: Tables, Chairs, and Desks. The company wants to maximize its monthly profit while considering:
- Limited resources (wood, labor hours, machine hours)
- Production capacity constraints
- Market demand limits
- Minimum production requirements

This is a classic **Linear Programming (LP)** problem that we'll solve using Python and the PuLP library.

## 1. Problem Setup & Data Definition

In [ ]:
# Install PuLP if not already installed
# !pip install pulp pandas matplotlib seaborn --break-system-packages

import pulp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(f"PuLP version: {pulp.__version__}")
print("Setup complete!")

### Define Business Parameters

In [ ]:
# Products
products = ['Table', 'Chair', 'Desk']

# Profit per unit (in dollars)
profit = {
    'Table': 120,
    'Chair': 80,
    'Desk': 150
}

# Resource requirements per unit
resource_requirements = pd.DataFrame({
    'Wood (sq ft)': {'Table': 10, 'Chair': 5, 'Desk': 15},
    'Labor (hours)': {'Table': 4, 'Chair': 2, 'Desk': 6},
    'Machine (hours)': {'Table': 2, 'Chair': 1, 'Desk': 3}
})

# Available resources per month
available_resources = {
    'Wood (sq ft)': 5000,
    'Labor (hours)': 2000,
    'Machine (hours)': 1200
}

# Market demand limits (maximum units that can be sold)
max_demand = {
    'Table': 200,
    'Chair': 400,
    'Desk': 150
}

# Minimum production requirements (contractual obligations)
min_production = {
    'Table': 50,
    'Chair': 100,
    'Desk': 30
}

print("\n=== Business Parameters ===")
print("\nProfit per Unit:")
print(pd.DataFrame([profit]).T.rename(columns={0: 'Profit ($)'}))

print("\nResource Requirements per Unit:")
print(resource_requirements)

print("\nAvailable Resources:")
print(pd.DataFrame([available_resources]).T.rename(columns={0: 'Available'}))

## 2. Mathematical Formulation

### Decision Variables
Let:
- $x_1$ = number of Tables to produce
- $x_2$ = number of Chairs to produce
- $x_3$ = number of Desks to produce

### Objective Function
**Maximize:** $Z = 120x_1 + 80x_2 + 150x_3$

### Constraints

**Resource Constraints:**
1. Wood: $10x_1 + 5x_2 + 15x_3 \leq 5000$
2. Labor: $4x_1 + 2x_2 + 6x_3 \leq 2000$
3. Machine: $2x_1 + 1x_2 + 3x_3 \leq 1200$

**Demand Constraints:**
4. $x_1 \leq 200$ (Tables)
5. $x_2 \leq 400$ (Chairs)
6. $x_3 \leq 150$ (Desks)

**Minimum Production:**
7. $x_1 \geq 50$
8. $x_2 \geq 100$
9. $x_3 \geq 30$

**Non-negativity:**
10. $x_1, x_2, x_3 \geq 0$

## 3. Model Implementation with PuLP

In [ ]:
# Create the LP problem
model = pulp.LpProblem("Furniture_Production_Optimization", pulp.LpMaximize)

# Decision variables
production = pulp.LpVariable.dicts(
    "Production",
    products,
    lowBound=0,
    cat='Integer'  # Integer production quantities
)

# Objective function: Maximize profit
model += pulp.lpSum([profit[p] * production[p] for p in products]), "Total_Profit"

# Resource constraints
for resource in resource_requirements.columns:
    model += (
        pulp.lpSum([resource_requirements.loc[p, resource] * production[p] for p in products]) 
        <= available_resources[resource],
        f"Constraint_{resource.replace(' ', '_').replace('(', '').replace(')', '')}"
    )

# Demand constraints (maximum production)
for p in products:
    model += production[p] <= max_demand[p], f"Max_Demand_{p}"

# Minimum production constraints
for p in products:
    model += production[p] >= min_production[p], f"Min_Production_{p}"

print("Model created successfully!")
print(f"\nNumber of variables: {len(production)}")
print(f"Number of constraints: {len(model.constraints)}")

### Display the Complete Model

In [ ]:
print("\n=== COMPLETE LINEAR PROGRAMMING MODEL ===")
print(model)

## 4. Solve the Optimization Problem

In [ ]:
# Solve the model
solver = pulp.PULP_CBC_CMD(msg=1)  # CBC solver with output
model.solve(solver)

# Check solution status
print(f"\n{'='*60}")
print(f"Solution Status: {pulp.LpStatus[model.status]}")
print(f"{'='*60}")

if model.status == pulp.LpStatusOptimal:
    print("✓ Optimal solution found!")
else:
    print("✗ No optimal solution found.")

## 5. Extract and Analyze Results

In [ ]:
# Extract optimal production quantities
optimal_production = {p: production[p].varValue for p in products}

# Calculate metrics
total_profit = pulp.value(model.objective)
total_revenue = sum([optimal_production[p] * profit[p] for p in products])

# Create results dataframe
results_df = pd.DataFrame({
    'Product': products,
    'Optimal Production': [optimal_production[p] for p in products],
    'Profit per Unit ($)': [profit[p] for p in products],
    'Total Profit ($)': [optimal_production[p] * profit[p] for p in products],
    'Min Required': [min_production[p] for p in products],
    'Max Demand': [max_demand[p] for p in products]
})

print("\n=== OPTIMAL PRODUCTION PLAN ===")
print(results_df.to_string(index=False))
print(f"\n{'='*60}")
print(f"MAXIMUM TOTAL PROFIT: ${total_profit:,.2f}")
print(f"{'='*60}")

### Resource Utilization Analysis

In [ ]:
# Calculate resource usage
resource_usage = {}
for resource in resource_requirements.columns:
    used = sum([resource_requirements.loc[p, resource] * optimal_production[p] for p in products])
    available = available_resources[resource]
    resource_usage[resource] = {
        'Used': used,
        'Available': available,
        'Remaining': available - used,
        'Utilization (%)': (used / available) * 100
    }

resource_df = pd.DataFrame(resource_usage).T

print("\n=== RESOURCE UTILIZATION ===")
print(resource_df.to_string())

# Identify binding constraints (fully utilized resources)
print("\n=== BINDING CONSTRAINTS (Bottlenecks) ===")
binding_resources = resource_df[resource_df['Utilization (%)'] >= 99.9]
if len(binding_resources) > 0:
    for resource in binding_resources.index:
        print(f"  ✓ {resource}: {binding_resources.loc[resource, 'Utilization (%)']:.1f}% utilized (BOTTLENECK)")
else:
    print("  No binding constraints found.")

### Constraint Analysis (Shadow Prices)

In [ ]:
# Extract shadow prices (dual values)
print("\n=== SHADOW PRICES (Marginal Values) ===")
print("Shadow price = Additional profit from 1 more unit of resource\n")

shadow_prices = []
for name, constraint in model.constraints.items():
    if constraint.pi is not None and constraint.pi != 0:
        shadow_prices.append({
            'Constraint': name,
            'Shadow Price ($)': constraint.pi,
            'Slack': constraint.slack
        })

if shadow_prices:
    shadow_df = pd.DataFrame(shadow_prices)
    shadow_df = shadow_df.sort_values('Shadow Price ($)', ascending=False)
    print(shadow_df.to_string(index=False))
else:
    print("No active constraints with shadow prices.")

## 6. Visualization of Results

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Optimal Production Quantities
ax1 = axes[0, 0]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars1 = ax1.bar(products, [optimal_production[p] for p in products], color=colors, alpha=0.8)
ax1.set_xlabel('Product', fontsize=12, fontweight='bold')
ax1.set_ylabel('Units', fontsize=12, fontweight='bold')
ax1.set_title('Optimal Production Quantities', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontweight='bold')

# 2. Profit Contribution by Product
ax2 = axes[0, 1]
profit_values = [optimal_production[p] * profit[p] for p in products]
bars2 = ax2.bar(products, profit_values, color=colors, alpha=0.8)
ax2.set_xlabel('Product', fontsize=12, fontweight='bold')
ax2.set_ylabel('Profit ($)', fontsize=12, fontweight='bold')
ax2.set_title('Profit Contribution by Product', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'${int(height):,}',
            ha='center', va='bottom', fontweight='bold')

# 3. Resource Utilization
ax3 = axes[1, 0]
resources = list(resource_usage.keys())
utilization = [resource_usage[r]['Utilization (%)'] for r in resources]
colors_util = ['#FF6B6B' if u >= 99 else '#4ECDC4' for u in utilization]
bars3 = ax3.barh(resources, utilization, color=colors_util, alpha=0.8)
ax3.set_xlabel('Utilization (%)', fontsize=12, fontweight='bold')
ax3.set_title('Resource Utilization Rate', fontsize=14, fontweight='bold')
ax3.axvline(x=100, color='red', linestyle='--', linewidth=2, label='100% Capacity')
ax3.legend()
ax3.grid(axis='x', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars3, utilization)):
    ax3.text(val + 1, bar.get_y() + bar.get_height()/2.,
            f'{val:.1f}%',
            va='center', fontweight='bold')

# 4. Production vs Constraints
ax4 = axes[1, 1]
x_pos = np.arange(len(products))
width = 0.25

ax4.bar(x_pos - width, [min_production[p] for p in products], width, 
        label='Minimum', color='#95E1D3', alpha=0.8)
ax4.bar(x_pos, [optimal_production[p] for p in products], width, 
        label='Optimal', color='#4ECDC4', alpha=0.8)
ax4.bar(x_pos + width, [max_demand[p] for p in products], width, 
        label='Maximum', color='#FF6B6B', alpha=0.8)

ax4.set_xlabel('Product', fontsize=12, fontweight='bold')
ax4.set_ylabel('Units', fontsize=12, fontweight='bold')
ax4.set_title('Production vs Constraints', fontsize=14, fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(products)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/home/claude/optimization_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved as 'optimization_results.png'")

## 7. Sensitivity Analysis

In [ ]:
# Sensitivity analysis: How does profit change with resource availability?
print("\n=== SENSITIVITY ANALYSIS ===")
print("Testing impact of ±20% change in resource availability\n")

sensitivity_results = []

for resource in available_resources.keys():
    for change_pct in [-20, -10, 0, 10, 20]:
        # Create new model
        test_model = pulp.LpProblem("Sensitivity_Test", pulp.LpMaximize)
        test_production = pulp.LpVariable.dicts("Prod", products, lowBound=0, cat='Integer')
        
        # Objective
        test_model += pulp.lpSum([profit[p] * test_production[p] for p in products])
        
        # Constraints with modified resource
        for r in resource_requirements.columns:
            available = available_resources[r]
            if r == resource:
                available = available * (1 + change_pct/100)
            test_model += pulp.lpSum([resource_requirements.loc[p, r] * test_production[p] 
                                     for p in products]) <= available
        
        # Other constraints
        for p in products:
            test_model += test_production[p] <= max_demand[p]
            test_model += test_production[p] >= min_production[p]
        
        # Solve
        test_model.solve(pulp.PULP_CBC_CMD(msg=0))
        
        if test_model.status == pulp.LpStatusOptimal:
            sensitivity_results.append({
                'Resource': resource,
                'Change (%)': change_pct,
                'New Availability': available_resources[resource] * (1 + change_pct/100),
                'Profit ($)': pulp.value(test_model.objective),
                'Profit Change (%)': ((pulp.value(test_model.objective) - total_profit) / total_profit) * 100
            })

sensitivity_df = pd.DataFrame(sensitivity_results)
print(sensitivity_df.to_string(index=False))

In [ ]:
# Visualize sensitivity analysis
fig, ax = plt.subplots(figsize=(12, 6))

for resource in available_resources.keys():
    data = sensitivity_df[sensitivity_df['Resource'] == resource]
    ax.plot(data['Change (%)'], data['Profit ($)'], marker='o', linewidth=2, 
           label=resource, markersize=8)

ax.set_xlabel('Resource Change (%)', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Profit ($)', fontsize=12, fontweight='bold')
ax.set_title('Sensitivity Analysis: Profit vs Resource Availability', 
            fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.axvline(x=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('/home/claude/sensitivity_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Sensitivity analysis plot saved!")

## 8. Key Insights & Business Recommendations

In [ ]:
print("\n" + "="*70)
print("KEY INSIGHTS & BUSINESS RECOMMENDATIONS")
print("="*70)

# Insight 1: Optimal production
print("\n1. OPTIMAL PRODUCTION PLAN:")
for p in products:
    print(f"   • {p}: {int(optimal_production[p])} units (${optimal_production[p] * profit[p]:,.0f} profit)")
print(f"   • Total Monthly Profit: ${total_profit:,.2f}")

# Insight 2: Bottlenecks
print("\n2. RESOURCE BOTTLENECKS:")
bottlenecks = resource_df[resource_df['Utilization (%)'] >= 95].sort_values('Utilization (%)', ascending=False)
if len(bottlenecks) > 0:
    for resource in bottlenecks.index:
        util = bottlenecks.loc[resource, 'Utilization (%)']
        print(f"   • {resource}: {util:.1f}% utilized - PRIMARY BOTTLENECK")
    print("   Recommendation: Invest in expanding these constrained resources.")
else:
    print("   • No critical bottlenecks identified.")

# Insight 3: Underutilized resources
print("\n3. UNDERUTILIZED RESOURCES:")
underutilized = resource_df[resource_df['Utilization (%)'] < 90].sort_values('Utilization (%)')
if len(underutilized) > 0:
    for resource in underutilized.index:
        util = underutilized.loc[resource, 'Utilization (%)']
        remaining = underutilized.loc[resource, 'Remaining']
        print(f"   • {resource}: {util:.1f}% utilized ({remaining:.0f} units unused)")
    print("   Recommendation: Consider reallocating or reducing excess capacity.")

# Insight 4: Most profitable product
print("\n4. PROFITABILITY ANALYSIS:")
profit_contribution = {p: optimal_production[p] * profit[p] for p in products}
most_profitable = max(profit_contribution, key=profit_contribution.get)
print(f"   • Highest profit contributor: {most_profitable} (${profit_contribution[most_profitable]:,.0f})")
print(f"   • Profit margin per unit: {most_profitable} = ${profit[most_profitable]}")

# Insight 5: Sensitivity to changes
print("\n5. SENSITIVITY TO RESOURCE CHANGES:")
for resource in available_resources.keys():
    data_10 = sensitivity_df[(sensitivity_df['Resource'] == resource) & 
                             (sensitivity_df['Change (%)'] == 10)]
    if not data_10.empty:
        profit_change = data_10['Profit Change (%)'].values[0]
        print(f"   • 10% increase in {resource}: {profit_change:+.1f}% profit change")

print("\n6. ACTION ITEMS:")
print("   ✓ Implement the optimal production plan immediately")
print("   ✓ Focus on expanding bottleneck resources for maximum profit impact")
print("   ✓ Monitor resource utilization monthly and adjust as needed")
print("   ✓ Review pricing strategy for low-margin products")
print("   ✓ Explore market expansion opportunities for high-profit items")
print("\n" + "="*70)

## 9. Summary Statistics

In [ ]:
# Create comprehensive summary
summary = {
    'Metric': [
        'Total Units Produced',
        'Total Profit',
        'Average Profit per Unit',
        'Resource Efficiency (avg)',
        'Constraints Met',
        'Optimization Status'
    ],
    'Value': [
        f"{sum(optimal_production.values()):.0f} units",
        f"${total_profit:,.2f}",
        f"${total_profit / sum(optimal_production.values()):.2f}",
        f"{resource_df['Utilization (%)'].mean():.1f}%",
        f"{len(model.constraints)}/{len(model.constraints)}",
        pulp.LpStatus[model.status]
    ]
}

summary_df = pd.DataFrame(summary)
print("\n=== EXECUTIVE SUMMARY ===")
print(summary_df.to_string(index=False))

# Save all results to CSV
results_df.to_csv('/home/claude/optimal_production_plan.csv', index=False)
resource_df.to_csv('/home/claude/resource_utilization.csv')
sensitivity_df.to_csv('/home/claude/sensitivity_analysis.csv', index=False)

print("\n✓ All results exported to CSV files")
print("  - optimal_production_plan.csv")
print("  - resource_utilization.csv")
print("  - sensitivity_analysis.csv")

## Conclusion

This notebook demonstrated a complete workflow for solving a business optimization problem using Linear Programming:

1. **Problem Formulation**: Defined decision variables, objective function, and constraints
2. **Model Implementation**: Used PuLP library to create and solve the LP model
3. **Solution Analysis**: Extracted optimal values and analyzed resource utilization
4. **Sensitivity Analysis**: Evaluated the impact of resource changes on profitability
5. **Business Insights**: Translated mathematical results into actionable recommendations

### Key Takeaways:
- Linear Programming is powerful for resource allocation and production planning
- Shadow prices help identify which constraints are most valuable to relax
- Sensitivity analysis provides insights for strategic decision-making
- The optimal solution maximizes profit while respecting all business constraints

### Extensions:
- Add more products or resources
- Include transportation or storage costs
- Model multi-period planning with inventory
- Incorporate uncertainty with stochastic programming
- Use integer programming for indivisible resources